# ids projekti, kivaa :---)

tästä alkaa suuri seikkailu. Olkaamme innokaita ja tehokkaita. 

In [224]:
import json
import pandas as pd
import numpy as np
import urllib.request as ur
import requests
from datetime import timedelta, date
from sklearn.model_selection import train_test_split as tts
from tpot import TPOTClassifier

In [392]:

def date_range(n):
    date_list = []

    for single_date in (date.today() - timedelta(n+1) for n in range(n)):
        date_list.append(single_date.strftime('%Y%m%d'))

    return(date_list)

In [393]:

def fetch_game_scoreboards():
    games_lists = []

    auth=('Enzine', 'Enzine')

    url = 'https://api.mysportsfeeds.com/v1.1/pull/nhl/2017-2018-regular/scoreboard.json?fordate={date}'
    
    DATE_RANGE = 100
    n=0
    for i in date_range(DATE_RANGE):
        if n%50 == 0:
            print(n)
        
        try:
            json = requests.get(url.format(date=i), auth=auth).json()
            json['scoreboard']['gameScore']
        except:
            continue
        
        games_lists.append(json['scoreboard']['gameScore'])
        
        n += 1

    return(games_lists)

In [248]:
#date_range(30)

In [394]:
game_data = fetch_game_scoreboards()

game_data_list = np.hstack(game_data)

0


Jos kotijoukkoe voittaa, winner = 1

In [395]:
print(game_data_list.shape)

(103,)


In [396]:
id_to_teamname = dict()

def parse_game(g):
    data = dict()
    

    data['away_team_id'] = g['game']['awayTeam']['ID']
    data['home_team_id'] = g['game']['homeTeam']['ID']
    
    try:
        data['home_score'] = g['homeScore']
    except:
        print(g['game']['ID'])
    data['away_score'] = g['awayScore']
    data['away_shots'] = g['awayShots']
    data['home_shots'] = g['awayShots']
    
    if g['game']['awayTeam']['Abbreviation'] not in id_to_teamname:
        id_to_teamname[g['game']['awayTeam']['Abbreviation']] = g['game']['awayTeam']['ID']
        
    if g['game']['homeTeam']['Abbreviation'] not in id_to_teamname:
        id_to_teamname[g['game']['homeTeam']['Abbreviation']] = g['game']['homeTeam']['ID']
    
    if data['home_score'] > data['away_score']:
        data['winner'] = 1
    else:
        data['winner'] = 0

    return(pd.Series(data))


In [496]:
df = pd.DataFrame()

for g in game_data_list:
    df = df.append(parse_game(g), ignore_index=True)
    
df['winner'] = df['winner'].astype('category')

In [402]:
print(game_data_list)

[ {'game': {'ID': '40868', 'scheduleStatus': 'Normal', 'originalDate': None, 'originalTime': None, 'delayedOrPostponedReason': None, 'date': '2017-10-19', 'time': '7:00PM', 'awayTeam': {'ID': '21', 'City': 'Vancouver', 'Name': 'Canucks', 'Abbreviation': 'VAN'}, 'homeTeam': {'ID': '11', 'City': 'Boston', 'Name': 'Bruins', 'Abbreviation': 'BOS'}, 'location': 'TD Garden'}, 'isUnplayed': 'false', 'isInProgress': 'false', 'isCompleted': 'true', 'awayScore': '3', 'awayShots': '29', 'homeScore': '6', 'homeShots': '35', 'periodSummary': {'period': [{'@number': '1', 'awayScore': '1', 'awayShots': '8', 'homeScore': '4', 'homeShots': '18'}, {'@number': '2', 'awayScore': '2', 'awayShots': '9', 'homeScore': '1', 'homeShots': '7'}, {'@number': '3', 'awayScore': '0', 'awayShots': '12', 'homeScore': '1', 'homeShots': '10'}]}}
 {'game': {'ID': '40869', 'scheduleStatus': 'Normal', 'originalDate': None, 'originalTime': None, 'delayedOrPostponedReason': None, 'date': '2017-10-19', 'time': '7:00PM', 'awayT

In [497]:
val = df['winner']
df = df.drop('winner', axis=1)

X_train, X_test, y_train, y_test = tts(df, val, train_size=0.75, test_size=0.25, random_state=42)

X_test.loc[:, 'away_score'] = -1
X_test.loc[:, 'home_score'] = -1
X_test.loc[:, 'away_shots'] = -1
X_test.loc[:, 'home_shots'] = -1

/Users/Jussi/vens/tf/lib/python3.6/site-packages/pandas/core/indexing.py:601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/Users/Jussi/vens/tf/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [457]:

tpot = TPOTClassifier(generations=30, population_size=30, verbosity=2, n_jobs=2)
tpot.fit(X_train, y_train)


Optimization Progress:   6%|▋         | 60/930 [00:15<06:42,  2.16pipeline/s]

Generation 1 - Current best internal CV score: 1.0


Optimization Progress:  10%|▉         | 90/930 [00:30<10:20,  1.35pipeline/s]

Generation 2 - Current best internal CV score: 1.0


Optimization Progress:  13%|█▎        | 120/930 [00:44<08:34,  1.58pipeline/s]

...

Generation 28 - Current best internal CV score: 1.0


Optimization Progress:  97%|█████████▋| 900/930 [03:11<00:09,  3.10pipeline/s]

Generation 29 - Current best internal CV score: 1.0


Generation 30 - Current best internal CV score: 1.0

Best pipeline: LinearSVC(input_matrix, C=1.0, dual=False, loss=squared_hinge, penalty=l2, tol=0.0001)


TPOTClassifier(config_dict={'sklearn.naive_bayes.GaussianNB': {}, 'sklearn.naive_bayes.BernoulliNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.naive_bayes.MultinomialNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.tree.DecisionT....45,
        0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,  0.9 ,
        0.95,  1.  ])}}}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        early_stop=None, generations=30, max_eval_time_mins=5,
        max_time_mins=None, mutation_rate=0.9, n_jobs=2, offspring_size=30,
        periodic_checkpoint_folder=None, population_size=30,
        random_state=None, scoring=None, subsample=1.0, verbosity=2,
        warm_start=False)

In [498]:

print(tpot.score(X_test, y_test))
#tpot.export('LinearSVC001.py')





0.423076923077


In [499]:
#print(id_to_teamname)

def predict_game(away, home):
    predict_data = dict()
    
    predict_data['away_team_id'] = id_to_teamname[away]
    predict_data['home_team_id'] = id_to_teamname[home]
    predict_data['home_score'] = -1
    predict_data['away_score'] = -1
    predict_data['away_shots'] = -1
    predict_data['home_shots'] = -1
    
    res = tpot.predict(pd.Series(predict_data).values.reshape(1, -1))
    
    print(pd.Series(predict_data).values.reshape(1, -1))
    
    winner = None
    
    if res[0] == 0:
        winner = away
    else:
        winner = home
    
    return(winner)


In [507]:
predict_game('VAN', 'BUF')

[[-1 -1 '21' -1 -1 '15']]


'VAN'

###### 20.10.17

VAN-BUF=VAN

SJS-NJD=SJS

WSH-DET=WSH

PIT-FLA=ERROR no FLA ????

MIN-WPG=ERROR no WPG ????

MTL-ANA=MTL
